<a href="https://www.kaggle.com/code/scr0ll0/modeling-with-validation?scriptVersionId=152606153" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

# Loading Data

Credit to Darek for the daigt-v2-train-dataset: https://www.kaggle.com/datasets/thedrcat/daigt-v2-train-dataset

In [2]:
train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
#train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sample = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

In [3]:
train_prompts

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [4]:
train

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
...,...,...,...,...,...
44863,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44864,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44865,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True
44866,"Dear Senator,\n\nI am writing to you today to ...",1,Does the electoral college work?,kingki19_palm,True


In [5]:
test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [6]:
sample

,id,generated
0,0000aaaa,0.1
1,1111bbbb,0.9
2,2222cccc,0.4


# Cleaning

In [7]:
train = train[['text', 'label']].copy()
train.rename(columns={'label': 'generated'}, inplace=True)
test = test.drop(columns=['id', 'prompt_id'])

In [8]:
dev_x = train.drop(columns='generated')
dev_y = train['generated']
#dev_x, dev_y = RandomOverSampler(random_state=0).fit_resample(dev_x, dev_y)
dev_x, dev_y = RandomUnderSampler(random_state=0).fit_resample(dev_x, dev_y)
#dev_x, dev_y = SMOTE(random_state=0).fit_resample(dev_x, dev_y)

In [9]:
#Tokenization
#vector = CountVectorizer(stop_words='english')
vector = TfidfVectorizer(stop_words='english')
vector.fit(dev_x['text'])
dev_x = pd.DataFrame(vector.transform(dev_x['text']).toarray())
test = pd.DataFrame(vector.transform(test['text']).toarray())

# Upsampling

In [10]:
#train_x, val_x, train_y, val_y = train_test_split(dev_x, dev_y, test_size=0.2, random_state=0, stratify=dev_y)

In [11]:
#Scaling
#scaler = StandardScaler()
#scaler.fit(train_x)
#train_x = scaler.transform(train_x)
#val_x = scaler.transform(val_x)

# Modeling

In [12]:
#Logistic Regression
#model = LogisticRegression()
#model.fit(train_x, train_y)
#y_pred = model.predict(val_x)
#roc_auc_score(val_y, y_pred)

# Testing

In [13]:
scaler = StandardScaler()
scaler.fit(dev_x)
dev_x = scaler.transform(dev_x)
test = scaler.transform(test)

In [14]:
model = LogisticRegression()
model.fit(dev_x, dev_y)
sample['generated'] = model.predict_proba(test)[:, 1]

In [15]:
sample['generated']

0    0.580814
1    0.698768
2    0.698768
Name: generated, dtype: float64

In [16]:
sample.to_csv('submission.csv', index=False)